In [ ]:
!pip install -q gradio
!pip install -q transformers
!pip install -q torchaudio
!pip install -q moviepy
!pip install -q pillow
!pip install -q requests
!pip install -q librosa
!pip install -q soundfile
!pip install -q accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.1 MB/s eta 0:00:00


In [1]:
import requests
import numpy as np
import torch
import pandas as pd
import os
import gradio as gr
import json
import time
from datetime import datetime
from moviepy.editor import VideoFileClip
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline, WhisperProcessor, WhisperForConditionalGeneration

class DescripteurVideoAutomatique:
    def __init__(self, intervalle=5, top_n=10, utiliser_cuda=True):
        self.intervalle = intervalle
        self.top_n = top_n
        self.utiliser_cuda = utiliser_cuda
        self.device = "cuda" if utiliser_cuda and torch.cuda.is_available() else "cpu"
        self.chemin_video = None
        self.scenes = []
        self.descriptions = []
        self.api_key_mistral = "JpPD7Mcbkn4kt9EASK8FyXKT0s8zdNn5"
        self.bibliotheque_path = "bibliotheque_videos.json"
        self.bibliotheque = self.charger_bibliotheque()

    def charger_bibliotheque(self):
        if os.path.exists(self.bibliotheque_path):
            try:
                with open(self.bibliotheque_path, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except:
                return []
        else:
            return []

    def sauvegarder_bibliotheque(self):
        with open(self.bibliotheque_path, 'w', encoding='utf-8') as f:
            json.dump(self.bibliotheque, f, ensure_ascii=False, indent=4)

    def generer_nom_descriptif(self, legendes, transcriptions):
        """Génère un nom descriptif à partir des légendes et transcriptions"""
        # Combinaison des 3 premières légendes
        nom_base = " - ".join([leg.split(".")[0] for leg in legendes[:3] if leg])
        # Limiter la longueur du nom
        if not nom_base:
            nom_base = "Vidéo sans description"
        return nom_base[:50] + "..." if len(nom_base) > 50 else nom_base

    def ajouter_a_bibliotheque(self, video_path, resultats, nom_personnalise=None):
        nom_fichier = os.path.basename(video_path)
        date_analyse = datetime.now().strftime("%d/%m/%Y %H:%M")

        # Extraction du premier moment intéressant
        moment_interessant = "Aucun moment intéressant trouvé"
        if resultats:
            top = max(resultats, key=lambda x: x["score"])
            moment_interessant = f"{self.formater_temps(top['debut'])}-{self.formater_temps(top['fin'])}"

        # Génération d'un nom descriptif si aucun nom personnalisé n'est fourni
        if not nom_personnalise:
            legendes = [r["legende"] for r in resultats[:3]] if resultats else []
            transcriptions = [r["transcription"] for r in resultats[:3]] if resultats else []
            nom_descriptif = self.generer_nom_descriptif(legendes, transcriptions)
        else:
            nom_descriptif = nom_personnalise

        # Création d'un identifiant unique basé sur le nom et la date
        video_id = f"{nom_fichier}_{int(time.time())}"

        # Ajout à la bibliothèque
        self.bibliotheque.append({
            "id": video_id,
            "nom": nom_fichier,
            "nom_descriptif": nom_descriptif,
            "chemin": video_path,
            "date_analyse": date_analyse,
            "moment_interessant": moment_interessant,
            "nombre_scenes": len(resultats) if resultats else 0
        })

        # Sauvegarde
        self.sauvegarder_bibliotheque()

        return self.bibliotheque

    def charger_modele(self):
        print("📦 Chargement des modèles...")
        # Modèle BLIP pour la description d'images
        self.processeur = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
        self.modele = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(self.device)


        # Modèle Whisper pour la transcription audio
        self.whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-small")
        self.whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(self.device)

        # Traducteur
        self.traducteur = pipeline("translation_en_to_fr", model="Helsinki-NLP/opus-mt-en-fr")
        print("✅ Modèles chargés avec succès!")

    def extraire_scenes(self):
        clip = VideoFileClip(self.chemin_video)
        duree = clip.duration
        self.scenes = [(debut, min(debut + self.intervalle, duree)) for debut in range(0, int(duree), self.intervalle)]

    def obtenir_image_milieu(self, debut, fin):
        clip = VideoFileClip(self.chemin_video).subclip(debut, fin)
        image = clip.get_frame((fin - debut) / 2)
        return Image.fromarray(image)

    def generer_legende(self, image):
        inputs = self.processeur(image, return_tensors="pt").to(self.device)
        sortie = self.modele.generate(**inputs)
        legende_en = self.processeur.decode(sortie[0], skip_special_tokens=True)
        legende_fr = self.traducteur(legende_en)[0]['translation_text']
        return legende_fr

    def extraire_audio(self, debut, fin, nom="scene_audio.wav"):
        try:
            clip = VideoFileClip(self.chemin_video).subclip(debut, fin)
            if clip.audio is not None:
                audio_path = os.path.join(os.getcwd(), nom)
                clip.audio.write_audiofile(audio_path, logger=None)
                return audio_path
            else:
                print(f"Avertissement: Pas d'audio dans la section {debut}-{fin}")
                return None
        except Exception as e:
            print(f"Erreur d'extraction audio: {str(e)}")
            return None

    def transcrire_audio_whisper(self, audio_path):
        if audio_path is None:
            return "Pas d'audio disponible pour cette section."

        try:
            import librosa
            import soundfile as sf

            # Chargement de l'audio
            audio, rate = librosa.load(audio_path, sr=16000)

            # Convertir en format attendu par Whisper
            input_features = self.whisper_processor(audio, sampling_rate=16000, return_tensors="pt").input_features.to(self.device)

            # Générer les tokens de prédiction
            predicted_ids = self.whisper_model.generate(input_features, language="fr", task="transcribe")

            # Décodage de la transcription
            transcription = self.whisper_processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

            if not transcription:
                return "Aucune transcription détectée."

            return transcription

        except Exception as e:
            return f"Erreur de transcription Whisper: {str(e)}"
        finally:
            # Nettoyage du fichier
            try:
                if os.path.exists(audio_path):
                    os.remove(audio_path)
            except:
                pass

    def demander_a_mistral(self, prompt, contexte=""):
        url = "https://api.mistral.ai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key_mistral}",
            "Content-Type": "application/json"
        }

        messages = [
            {"role": "system", "content": "Vous êtes un assistant spécialisé dans l'analyse vidéo. Répondez en français."},
            {"role": "user", "content": f"Contexte sur la vidéo: {contexte}\n\nQuestion: {prompt}"}
        ]

        data = {
            "model": "mistral-large-latest",
            "messages": messages,
            "temperature": 0.7,
            "max_tokens": 800
        }

        try:
            response = requests.post(url, headers=headers, json=data)
            response.raise_for_status()
            result = response.json()
            return result["choices"][0]["message"]["content"]
        except Exception as e:
            return f"Erreur lors de la communication avec Mistral: {str(e)}"

    def decrire_video(self):
        self.extraire_scenes()
        self.descriptions = []

        for debut, fin in self.scenes:
            print(f"Analyse de la scène {self.formater_temps(debut)}-{self.formater_temps(fin)}...")

            # Génération de la légende visuelle
            image = self.obtenir_image_milieu(debut, fin)
            legende = self.generer_legende(image)

            # Extraction et transcription audio avec Whisper
            audio_path = self.extraire_audio(debut, fin, nom=f"audio_{int(debut)}.wav")
            transcription = self.transcrire_audio_whisper(audio_path)

            # Calcul du score (pondération ajustée pour favoriser les scènes avec audio)
            score = len(legende) * 0.05 + len(transcription) * 0.15 + np.random.rand() * 0.3

            self.descriptions.append({
                "debut": debut,
                "fin": fin,
                "legende": legende,
                "transcription": transcription,
                "score": score
            })

    def obtenir_meilleures_scenes(self):
        return sorted(self.descriptions, key=lambda x: x["score"], reverse=True)[:self.top_n]

    def formater_temps(self, secondes):
        return f"{int(secondes // 60):02d}:{int(secondes % 60):02d}"

    def trouver_moment_interessant(self, scenes):
        if not scenes:
            return "Aucun moment intéressant trouvé"

        top = max(scenes, key=lambda x: x["score"])
        return f"Le moment le plus intéressant se trouve entre {self.formater_temps(top['debut'])} et {self.formater_temps(top['fin'])}"

    def analyser_video(self, video_path, nom_personnalise=None):
        if not video_path:
            return None, "Veuillez télécharger une vidéo.", None

        # Sauvegarde du chemin de la vidéo
        self.chemin_video = video_path

        # Chargement du modèle si nécessaire
        if not hasattr(self, 'modele') or not hasattr(self, 'whisper_model'):
            self.charger_modele()

        # Analyse de la vidéo
        try:
            self.decrire_video()
            meilleures_scenes = self.obtenir_meilleures_scenes()
            moment_interessant = self.trouver_moment_interessant(meilleures_scenes)

            # Création d'un DataFrame pour l'affichage
            df = pd.DataFrame([{
                "Rang": i+1,
                "Début": self.formater_temps(s["debut"]),
                "Fin": self.formater_temps(s["fin"]),
                "Description": s["legende"],
                "Transcription audio": s["transcription"]
            } for i, s in enumerate(meilleures_scenes)])

            # Obtenir le nom de fichier de la vidéo pour l'affichage
            nom_fichier = os.path.basename(video_path)

            # Ajout à la bibliothèque
            bibliotheque_mise_a_jour = self.ajouter_a_bibliotheque(video_path, self.descriptions, nom_personnalise)

            return df, f"Vidéo: {nom_fichier}\n{moment_interessant}", bibliotheque_mise_a_jour

        except Exception as e:
            return None, f"Erreur lors de l'analyse: {str(e)}", None

    def analyser_et_demander(self, video_path, prompt, nom_personnalise=None):
        df, message, bibliotheque = self.analyser_video(video_path, nom_personnalise)

        if df is None:
            return None, message, None, bibliotheque

        # Préparation du contexte pour Mistral
        contexte = "Informations sur les scènes principales: "
        for i, row in df.head(3).iterrows():
            contexte += f"\nScène {row['Début']}-{row['Fin']}: {row['Description']}. Audio: {row['Transcription audio']}"

        # Interrogation de Mistral
        reponse_mistral = self.demander_a_mistral(prompt, contexte)

        return df, message, reponse_mistral, bibliotheque

# Interface Gradio
def creer_interface():
    # Création d'un descripteur partagé
    descr = DescripteurVideoAutomatique()

    # Fonctions pour la manipulation de la bibliothèque
    def charger_bibliotheque():
        return pd.DataFrame([
            {"Index": i, "ID": item["id"], "Nom": item["nom"],
             "Nom descriptif": item.get("nom_descriptif", ""),
             "Date d'analyse": item["date_analyse"],
             "Moment intéressant": item["moment_interessant"],
             "Nombre de scènes": item["nombre_scenes"]}
            for i, item in enumerate(descr.charger_bibliotheque())
        ])

    def supprimer_de_bibliotheque(index):
        try:
            index = int(index)
            if 0 <= index < len(descr.bibliotheque):
                descr.bibliotheque.pop(index)
                descr.sauvegarder_bibliotheque()
            return charger_bibliotheque()
        except:
            return charger_bibliotheque()

    def ajouter_video_a_bibliotheque(video, nom_descriptif):
        if not video:
            return "Veuillez télécharger une vidéo.", charger_bibliotheque()

        try:
            # Analyse simplifiée (juste pour l'extraction des scènes)
            descr.chemin_video = video
            # S'assurer que les modèles sont chargés
            if not hasattr(descr, 'modele') or not hasattr(descr, 'whisper_model'):
                descr.charger_modele()

            descr.extraire_scenes()
            # Analyse de quelques scènes clés seulement pour un traitement plus rapide
            scenes_echantillon = descr.scenes[:3]
            descriptions = []

            for debut, fin in scenes_echantillon:
                image = descr.obtenir_image_milieu(debut, fin)
                legende = descr.generer_legende(image)
                descriptions.append({
                    "debut": debut,
                    "fin": fin,
                    "legende": legende,
                    "transcription": "",
                    "score": len(legende) * 0.05 + np.random.rand() * 0.3
                })

            # Ajout à la bibliothèque avec le nom personnalisé
            descr.ajouter_a_bibliotheque(video, descriptions, nom_descriptif or None)
            return f"Vidéo ajoutée avec succès: {os.path.basename(video)}", charger_bibliotheque()

        except Exception as e:
            return f"Erreur lors de l'ajout: {str(e)}", charger_bibliotheque()

    # Fonction d'analyse pour une vidéo avec prompt
    def analyser_avec_prompt(video, prompt, nom_personnalise):
        if not video:
            return None, "Veuillez télécharger une vidéo.", "", None
        return descr.analyser_et_demander(video, prompt, nom_personnalise)

    # Style CSS personnalisé pour la barre de progrès type Canva
    css_personnalise = """
    .video-player {
        max-width: 640px;
        max-height: 360px;
        margin: 0 auto;
    }

    .progress-bar-container {
        width: 100%;
        height: 10px;
        background-color: #ddd;
        border-radius: 5px;
        overflow: hidden;
        margin-top: -15px;
        position: relative;
        z-index: 10;
    }

    .progress-bar {
        height: 100%;
        background-color: #ff5722;
        width: 0%;
        transition: width 0.3s;
    }

    .time-display {
        font-size: 14px;
        margin-top: 5px;
        text-align: center;
    }

    .video-card {
        border: 1px solid #ddd;
        border-radius: 10px;
        padding: 10px;
        margin-bottom: 15px;
        background-color: #f9f9f9;
    }

    .bibliotheque-item {
        padding: 10px;
        border-bottom: 1px solid #eee;
        border-radius: 5px;
        margin-bottom: 5px;
    }

    .bibliotheque-item:hover {
        background-color: #f0f0f0;
    }
    """

    # Création de l'interface principale
    with gr.Blocks(css=css_personnalise, title="Descripteur Automatique de Vidéo") as interface:
        gr.Markdown("# Descripteur Automatique de Vidéo avec Whisper & Mistral AI")

        with gr.Tabs():
            # Onglet Analyse de Vidéo
            with gr.TabItem("Analyse de Vidéo"):
                with gr.Row():
                    with gr.Column(scale=2):
                        video_input = gr.Video(label="Téléchargez votre vidéo", elem_classes=["video-player"])
                        nom_descriptif_analyse = gr.Textbox(label="Nom descriptif de la vidéo (optionnel)", placeholder="Laissez vide pour génération automatique")
                        gr.HTML("""
                        <div class="progress-bar-container">
                            <div class="progress-bar" id="video-progress"></div>
                        </div>
                        <div class="time-display" id="time-display">00:00 / 00:00</div>
                        <script>
                            document.addEventListener('DOMContentLoaded', function() {
                                setTimeout(function() {
                                    const videoElements = document.querySelectorAll('video');
                                    videoElements.forEach(function(video) {
                                        const progressBar = document.getElementById('video-progress');
                                        const timeDisplay = document.getElementById('time-display');

                                        video.addEventListener('timeupdate', function() {
                                            const progress = (video.currentTime / video.duration) * 100;
                                            progressBar.style.width = progress + '%';

                                            const currentMinutes = Math.floor(video.currentTime / 60);
                                            const currentSeconds = Math.floor(video.currentTime % 60);
                                            const totalMinutes = Math.floor(video.duration / 60);
                                            const totalSeconds = Math.floor(video.duration % 60);

                                            timeDisplay.textContent =
                                                `${currentMinutes.toString().padStart(2, '0')}:${currentSeconds.toString().padStart(2, '0')} /
                                                ${totalMinutes.toString().padStart(2, '0')}:${totalSeconds.toString().padStart(2, '0')}`;
                                        });
                                    });
                                }, 1000);
                            });
                        </script>
                        """)

                    with gr.Column(scale=2):
                        prompt_input = gr.Textbox(label="Posez une question sur cette vidéo", placeholder="Ex: Quels sont les points clés de cette vidéo?", lines=3)
                        analyser_btn = gr.Button("Analyser la vidéo", variant="primary")
                        gr.Markdown("### Réponse de Mistral AI")
                        reponse_mistral = gr.Textbox(label="", placeholder="La réponse apparaîtra ici après l'analyse...", lines=6)

                with gr.Row():
                    with gr.Column():
                        gr.Markdown("### Moments clés")
                        resultats_df = gr.DataFrame(label="", headers=["Rang", "Début", "Fin", "Description", "Transcription audio"])
                        moment_interessant = gr.Textbox(label="Moment le plus intéressant")

                # Traitement du clic sur le bouton d'analyse
                analyser_btn.click(
                    fn=analyser_avec_prompt,
                    inputs=[video_input, prompt_input, nom_descriptif_analyse],
                    outputs=[resultats_df, moment_interessant, reponse_mistral, gr.State()]
                )

            # Onglet Bibliothèque de Vidéos
            with gr.TabItem("Bibliothèque de Vidéos"):
                gr.Markdown("## Vos vidéos analysées")

                with gr.Row():
                    bibliotheque_btn = gr.Button("Actualiser la bibliothèque")

                with gr.Row():
                    bibliotheque_list = gr.DataFrame(
                        headers=["Index", "ID", "Nom", "Nom descriptif", "Date d'analyse", "Moment intéressant", "Nombre de scènes"],
                        label="Vidéos sauvegardées"
                    )

                with gr.Row():
                    with gr.Column(scale=1):
                        index_suppression = gr.Number(label="Index de la vidéo à supprimer", precision=0)
                        supprimer_btn = gr.Button("Supprimer de la bibliothèque")

                    with gr.Column(scale=2):
                        gr.Markdown("### Ajouter une vidéo à la bibliothèque")
                        video_ajout = gr.Video(label="Vidéo à ajouter")
                        nom_descriptif = gr.Textbox(label="Nom descriptif", placeholder="Entrez un nom descriptif pour cette vidéo")
                        ajouter_btn = gr.Button("Ajouter à la bibliothèque", variant="primary")
                        resultat_ajout = gr.Textbox(label="Résultat de l'ajout")

                # Chargement initial de la bibliothèque
                interface.load(
                    fn=charger_bibliotheque,
                    inputs=None,
                    outputs=bibliotheque_list
                )

                # Actualisation de la bibliothèque
                bibliotheque_btn.click(
                    fn=charger_bibliotheque,
                    inputs=None,
                    outputs=bibliotheque_list
                )

                # Suppression d'une vidéo de la bibliothèque
                supprimer_btn.click(
                    fn=supprimer_de_bibliotheque,
                    inputs=index_suppression,
                    outputs=bibliotheque_list
                )

                # Ajout d'une vidéo à la bibliothèque
                ajouter_btn.click(
                    fn=ajouter_video_a_bibliotheque,
                    inputs=[video_ajout, nom_descriptif],
                    outputs=[resultat_ajout, bibliotheque_list]
                )

    return interface

# Pour une utilisation dans Google Colab
if __name__ == "__main__":
    # Installation des dépendances si nécessaire
    try:
        import gradio
    except ImportError:
        print("🔧 Installation des dépendances nécessaires...")
        !pip install gradio transformers moviepy pillow requests librosa soundfile -q

    try:
        from transformers import WhisperProcessor
    except:
        !pip install transformers[torch] -q

    # Importation des bibliothèques après installation
    import gradio as gr
    import numpy as np
    import torch
    from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
    from transformers import WhisperProcessor, WhisperForConditionalGeneration
    from moviepy.editor import VideoFileClip

    # Lancement de l'interface
    interface = creer_interface()
    interface.launch(debug=True, share=True)

ModuleNotFoundError: No module named 'gradio'